In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/processed_movie_data.csv", delimiter=';', encoding='utf-8')
df

,budget,genres,imdb_id,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,overview_length,sentiment,year,month,weekday,genre_id
0,30000000.0,Animation,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1.0,1.0,1995-10-30,373554033.0,81.0,1.0,7.7,5415.0,303,0.112121,1995,10,0,0
1,65000000.0,Adventure,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,3.0,1.0,1995-12-15,262797249.0,104.0,2.0,6.9,2413.0,395,-0.218750,1995,12,4,1
2,16000000.0,Comedy,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1.0,1.0,1995-12-22,81452156.0,127.0,1.0,6.1,34.0,270,0.600000,1995,12,4,2
3,60000000.0,Action,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,3.0,1.0,1995-12-15,187436818.0,170.0,2.0,7.7,1886.0,335,-0.053571,1995,12,4,3
4,35000000.0,Action,tt0114576,Sudden Death,International action superstar Jean Claude Van...,5.231580,3.0,1.0,1995-12-22,64350171.0,106.0,1.0,5.5,174.0,448,-0.116667,1995,12,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680,2196531.0,Comedy,tt0453365,FC Venus,"Pete is a football enthusiast, who plays as a ...",0.947509,1.0,1.0,2005-12-30,2411594.0,107.0,1.0,5.6,10.0,669,0.102083,2005,12,4,2
4681,12000000.0,Action,tt0479751,சிவாஜி,Corrupt police and politicians target a comput...,1.323587,1.0,1.0,2007-06-14,19000000.0,185.0,2.0,6.9,25.0,117,-0.055556,2007,6,3,3
4682,800000.0,Comedy,tt0933361,Dikari,"The sea, August, interesting and simple people...",0.903061,0.0,0.0,2006-11-23,1328612.0,100.0,1.0,5.8,6.0,378,0.380000,2006,11,3,2
4683,2000000.0,Romance,tt1718881,Про любоff,"У девушки Даши, приехавшей с подругой «покорят...",0.121844,1.0,1.0,2010-09-30,1268793.0,107.0,1.0,4.0,3.0,534,0.000000,2010,9,3,9


In [3]:
from utils.kohonen.kohonen import build_kohonen_net
from utils.kohonen.neuron_neighbours import prop2iters_neighbour_radius, relu_like_neighbour_radius, exp_neighbour_radius
from utils.kohonen.neuron_lr import inv2iter_learning_rate, fixed_learning_rate

In [4]:
cols = ["budget", "popularity", "production_companies", 
        "production_countries", "revenue", "runtime", 
        "spoken_languages", "vote_average", "vote_count",
        "overview_length", "sentiment", "year",
        "month", "weekday"]

In [5]:
seed_range = 2

In [6]:
to_normalize = cols
for i in range(len(to_normalize)):
    df[to_normalize[i]] = (df[to_normalize[i]] - df[to_normalize[i]].mean()) / df[to_normalize[i]].std()
df

,budget,genres,imdb_id,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,overview_length,sentiment,year,month,weekday,genre_id
0,-0.030242,Animation,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",2.297018,-0.888981,-0.467054,1995-10-30,1.813769,-1.351069,-0.528057,1.573598,3.973387,-0.003497,0.312006,-0.291643,0.885889,-2.399597,-1.326392
1,0.850490,Adventure,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,1.420062,0.025974,-0.467054,1995-12-15,1.108865,-0.287850,0.534179,0.686010,1.436552,0.570333,-1.197489,-0.291643,1.475559,0.619027,-1.021418
2,-0.382535,Comedy,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",-0.919488,-0.888981,-0.467054,1995-12-22,-0.045295,0.775369,-0.528057,-0.201578,-0.573818,-0.209328,2.537798,-0.291643,1.475559,0.619027,-0.716444
3,0.724671,Action,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",1.581780,0.025974,-0.467054,1995-12-15,0.629238,2.763126,0.534179,1.573598,0.991211,0.196096,-0.443914,-0.291643,1.475559,0.619027,-0.411471
4,0.095577,Action,tt0114576,Sudden Death,International action superstar Jean Claude Van...,-0.675489,0.025974,-0.467054,1995-12-22,-0.154140,-0.195396,-0.528057,-0.867269,-0.455512,0.900909,-0.731766,-0.291643,1.475559,0.619027,-0.411471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680,-0.729883,Comedy,tt0453365,FC Venus,"Pete is a football enthusiast, who plays as a ...",-1.437329,-0.888981,-0.467054,2005-12-30,-0.548344,-0.149169,-0.528057,-0.756321,-0.594100,2.279350,0.266211,0.343454,1.475559,0.619027,-0.716444
4681,-0.483190,Action,tt0479751,சிவாஜி,Corrupt police and politicians target a comput...,-1.370451,-0.888981,-0.467054,2007-06-14,-0.442768,3.456530,0.534179,0.686010,-0.581424,-1.163633,-0.452966,0.470473,-0.293450,-0.135629,-0.411471
4682,-0.765025,Comedy,tt0933361,Dikari,"The sea, August, interesting and simple people...",-1.445233,-1.346458,-1.755714,2006-11-23,-0.555237,-0.472758,-0.528057,-0.534424,-0.597480,0.464299,1.534118,0.406964,1.180724,-0.135629,-0.716444
4683,-0.734828,Romance,tt1718881,Про любоff,"У девушки Даши, приехавшей с подругой «покорят...",-1.584158,-0.888981,-0.467054,2010-09-30,-0.555617,-0.149169,-0.528057,-2.531497,-0.600015,1.437316,-0.199512,0.661002,0.591054,-0.135629,1.418372


In [7]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
rectangular



0 13277.608545778
max:  1 13405.956780884237
range:  128.34823510623755


In [8]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
rectangular

0 15510.887713335533
max:  1 15528.345851861972
range:  17.458138526439143


In [9]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
rectangular

0 13317.160943663652
1 13295.134782718813
max:  0 13317.160943663652
range:  22.02616094483892


In [10]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
rectangular

0 14076.641448168544
max:  1 14113.539603616564
range:  36.89815544801968


In [11]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
rectangular

0 13294.967036768203
max:  1 13322.846045890634
range:  27.879009122430944


In [12]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
rectangular

0 15494.017652779947
max:  1 15613.85786421069
range:  119.84021143074278


In [13]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
rectangular

0 13322.05480427807
1 13248.00338321429
max:  0 13322.05480427807
range:  74.05142106377934


In [14]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
rectangular

0 14179.034741059406
1 14153.827990728241
max:  0 14179.034741059406
range:  25.206750331164585


In [15]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
rectangular

0 13350.180306687473
max:  1 13466.174088388792
range:  115.99378170131968


In [16]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
rectangular

0 15570.497581543828
1 15550.889338554314
max:  0 15570.497581543828
range:  19.60824298951411


In [17]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
rectangular

0 13394.988785142188
1 13380.53479637482
max:  0 13394.988785142188
range:  14.453988767367264


In [18]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
rectangular

0 14501.092104199564
max:  1 14510.280433919572
range:  9.18832972000746


In [19]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
rectangular

0 13379.527598938646
1 13358.45046194326
max:  0 13379.527598938646
range:  21.077136995387264


In [20]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
rectangular

0 15535.298992053855
1 15456.189710185961
max:  0 15535.298992053855
range:  79.10928186789351


In [21]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
rectangular

0 13278.397353008724
max:  1 13281.655783124672
range:  3.2584301159477036


In [22]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
rectangular

0 14364.322715918332
max:  1 14620.106782192632
range:  255.78406627430013


In [23]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
hexagonal

0 13523.196351199036
max:  1 13697.214929092772
range:  174.0185778937357


In [24]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
hexagonal

0 15560.227043634353
max:  1 15591.768377408061
range:  31.541333773708175


In [25]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
hexagonal

0 13471.484190498406
max:  1 13480.775923291947
range:  9.291732793541087


In [26]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
hexagonal

0 14101.75428771957
1 14077.428436304632
max:  0 14101.75428771957
range:  24.32585141493837


In [27]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
hexagonal

0 13436.140593547108
max:  1 13475.389311747618
range:  39.24871820050976


In [28]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
hexagonal

0 15548.898367681431
max:  1 15601.22845702064
range:  52.330089339207916


In [29]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
hexagonal

0 13422.46071112404
max:  1 13457.619962440027
range:  35.159251315986694


In [30]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
hexagonal

0 14122.20172705813
max:  1 14154.788831763199
range:  32.58710470506958


In [31]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
hexagonal

0 13563.65076343272
max:  1 13611.515414765932
range:  47.86465133321144


In [32]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
hexagonal

0 15660.73667817453
1 15562.169323913451
max:  0 15660.73667817453
range:  98.56735426107844


In [33]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
hexagonal

0 13473.550468839445
1 13401.603285205647
max:  0 13473.550468839445
range:  71.94718363379798


In [34]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
hexagonal

0 14468.07281782651
max:  1 14498.923251601977
range:  30.850433775465717


In [35]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
hexagonal

0 13503.832907289023
max:  1 13561.026677952814
range:  57.19377066379093


In [36]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
hexagonal

0 15583.276724804264
max:  1 15606.868397837352
range:  23.591673033088227


In [37]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
hexagonal

0 13483.34956296945
1 13458.588805226684
max:  0 13483.34956296945
range:  24.760757742766145


In [38]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
hexagonal

0 14361.763694252852
max:  1 14617.709776108426
range:  255.94608185557445


In [39]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
rectangular

0 14178.935145189966
max:  1 14211.013561253503
range:  32.07841606353759


In [40]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
rectangular

0 15683.86585943536
1 15533.397526884288
max:  0 15683.86585943536
range:  150.4683325510723


In [41]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
rectangular

0 14265.99533559505
1 14175.11714549019
max:  0 14265.99533559505
range:  90.87819010485873


In [42]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
rectangular

0 14659.599657588607
max:  1 14677.744970067592
range:  18.145312478984124


In [43]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
rectangular

0 14142.048330408066
max:  1 14328.545112655771
range:  186.4967822477047


In [44]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
rectangular

0 15721.181588351024
1 15602.231734646932
max:  0 15721.181588351024
range:  118.94985370409267


In [45]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
rectangular

0 14177.246611179526
max:  1 14310.727387899135
range:  133.4807767196089


In [46]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
rectangular

0 14788.419674072231
1 14599.220195886048
max:  0 14788.419674072231
range:  189.19947818618311


In [47]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
rectangular

0 14208.524170142293
1 14155.628195169524
max:  0 14208.524170142293
range:  52.89597497276918


In [48]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
rectangular

0 15608.672204524184
max:  1 15647.467190083746
range:  38.79498555956161


In [49]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
rectangular

0 14197.65779583574
max:  1 14277.334762005852
range:  79.67696617011279


In [50]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
rectangular

0 14573.315606956394
max:  1 14796.84855265536
range:  223.5329456989657


In [51]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
rectangular

0 14251.184237882286
1 14230.1412569858
max:  0 14251.184237882286
range:  21.04298089648546


In [52]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
rectangular

0 15664.919570360848
max:  1 15669.116409046277
range:  4.196838685429611


In [53]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
rectangular

0 14159.500157782675
1 14098.501899527146
max:  0 14159.500157782675
range:  60.998258255529436


In [54]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
rectangular

0 14826.45489010657
1 14772.321837793565
max:  0 14826.45489010657
range:  54.133052313005464


In [55]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
hexagonal

0 14240.911158798659
max:  1 14330.368610490194
range:  89.45745169153452


In [56]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
hexagonal

0 15798.330287891797
1 15518.579764640746
max:  0 15798.330287891797
range:  279.75052325105025


In [57]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
hexagonal

0 14329.657900896946
1 14253.852488147435
max:  0 14329.657900896946
range:  75.80541274951065


In [58]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
hexagonal

0 14671.576054949754
max:  1 14683.66878667843
range:  12.092731728676881


In [59]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
hexagonal

0 14309.877582130683
max:  1 14349.047925649773
range:  39.17034351909024


In [60]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
hexagonal

0 15645.229403586054
1 15591.564262881779
max:  0 15645.229403586054
range:  53.66514070427547


In [61]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
hexagonal

0 14212.57548893795
max:  1 14336.604148773531
range:  124.0286598355815


In [62]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
hexagonal

0 14768.93477131767
1 14575.043490472885
max:  0 14768.93477131767
range:  193.8912808447858


In [63]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
hexagonal

0 14307.89999103679
1 14272.127441959183
max:  0 14307.89999103679
range:  35.77254907760653


In [64]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
hexagonal

0 15601.702419852461
max:  1 15612.886267225002
range:  11.183847372540185


In [65]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
hexagonal

0 14240.412758005348
max:  1 14300.947270179899
range:  60.5345121745504


In [66]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
hexagonal

0 14673.787477458665
max:  1 14859.292595426366
range:  185.50511796770115


In [67]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
hexagonal

0 14285.909419713687
1 14280.17917791411
max:  0 14285.909419713687
range:  5.730241799577925


In [68]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
hexagonal

0 15597.10087435797
max:  1 15651.361494974668
range:  54.26062061669836


In [69]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
hexagonal

0 14204.401761752768
max:  1 14212.633306647629
range:  8.231544894861145


In [70]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
hexagonal

0 14802.578175029996
1 14801.102368533895
max:  0 14802.578175029996
range:  1.4758064961006312


In [71]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
rectangular

0 12801.762000290817
max:  1 12864.316322496938
range:  62.55432220612056


In [72]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
rectangular

0 15537.91778355052
1 15465.798684634065
max:  0 15537.91778355052
range:  72.11909891645519


In [73]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
rectangular

0 12589.72433198237
max:  1 12695.37682533758
range:  105.65249335521003


In [74]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
rectangular

0 13673.409725850946
1 13562.952348340737
max:  0 13673.409725850946
range:  110.45737751020897


In [75]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
rectangular

0 12699.46888383182
max:  1 12783.04801048632
range:  83.57912665450021


In [76]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
rectangular

0 15647.024427649734
1 15432.346382134538
max:  0 15647.024427649734
range:  214.67804551519657


In [77]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
rectangular

0 12620.458434285587
max:  1 12693.877757541022
range:  73.4193232554353


In [78]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
rectangular

0 13723.492789566388
1 13690.025908137599
max:  0 13723.492789566388
range:  33.46688142878884


In [79]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
rectangular

0 12804.306478436336
max:  1 12944.109744644331
range:  139.80326620799497


In [80]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
rectangular

0 15459.708310591217
max:  1 15476.95268045075
range:  17.244369859532526


In [81]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
rectangular

0 12724.870041829221
1 12699.897160459257
max:  0 12724.870041829221
range:  24.972881369963943


In [82]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
rectangular

0 14183.877201526304
max:  1 14329.864982750789
range:  145.98778122448493


In [83]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
rectangular

0 12824.74592656601
1 12756.869898434192
max:  0 12824.74592656601
range:  67.87602813181911


In [84]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
rectangular



0 15591.814262229613
1 15469.007237222813
max:  0 15591.814262229613
range:  122.80702500679945


In [85]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
rectangular

0 12689.808284538934
max:  1 12714.117151121312
range:  24.30886658237796


In [86]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
rectangular

0 14098.513095374963
max:  1 14208.659065315487
range:  110.14596994052408


In [87]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
hexagonal

0 12857.383156530623
max:  1 12939.76910144261
range:  82.38594491198637


In [88]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
hexagonal

0 15571.605841397048
1 15496.995950348217
max:  0 15571.605841397048
range:  74.60989104883083


In [89]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
hexagonal

0 12808.769723138408
max:  1 12907.365291056527
range:  98.59556791811883


In [90]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
hexagonal

0 13637.002888768273
1 13510.200203870689
max:  0 13637.002888768273
range:  126.80268489758419


In [91]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
hexagonal

0 12905.16479460163
max:  1 12995.422798305513
range:  90.25800370388424


In [92]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
hexagonal

0 15714.795352857705
1 15429.908873553468
max:  0 15714.795352857705
range:  284.8864793042376


In [93]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
hexagonal

0 12756.599665772306
max:  1 12836.269427310817
range:  79.66976153851101


In [94]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
hexagonal

0 13648.941354633733
1 13638.500392824533
max:  0 13648.941354633733
range:  10.440961809199507


In [95]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
hexagonal

0 12894.89257388964
max:  1 13025.033044788734
range:  130.140470899094


In [96]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
hexagonal

0 15468.051854914585
1 15452.476712477095
max:  0 15468.051854914585
range:  15.575142437490285


In [97]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
hexagonal

0 12805.265314975015
max:  1 12829.039727629235
range:  23.774412654220214


In [98]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
hexagonal

0 14200.310567794106
max:  1 14345.525772505036
range:  145.21520471093027


In [99]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
hexagonal

0 12829.806825647842
max:  1 12927.797036078078
range:  97.9902104302364


In [100]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
hexagonal

0 15452.363739488246
max:  1 15486.810551225459
range:  34.446811737212556


In [101]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
hexagonal

0 12865.057946129258
1 12779.349754127672
max:  0 12865.057946129258
range:  85.70819200158621


In [102]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
hexagonal

0 14022.94892052642
max:  1 14219.205286872591
range:  196.2563663461715
